In [1]:
import sys
from all_datasets.COOS_dataset import COOSDataset
from all_datasets.FMoW_dataset import FMoWDataset
import numpy as np
import matplotlib.pyplot as plt
import h5py
from get_clip_embeddings import get_embeddings, get_dataloader, preprocess_images, preprocess_texts, fix_embedding
import torch

/data/vision/beery/scratch/neha/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/data/vision/beery/scratch/neha/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
from transformers import CLIPProcessor, CLIPModel
dataloader = get_dataloader("FMoW","train")
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

def get_embeddings_1(data_loader, clip_processor, clip_model):
    embeddings_list = []
    i=0
    for images, texts, labels, uids in data_loader:
        image_inputs = preprocess_images(images, clip_processor)
        text_inputs = preprocess_texts(texts, clip_processor)
        print("preprocessed")
        with torch.no_grad():
            image_features = clip_model.get_image_features(**image_inputs)
            text_features = clip_model.get_text_features(**text_inputs)
            image_embeddings = image_features / image_features.norm(dim=-1, keepdim=True)
            text_embeddings = text_features / text_features.norm(dim=-1, keepdim=True)
            similarity_scores = torch.nn.functional.cosine_similarity(image_embeddings, text_embeddings, dim=-1)
        
            for img_emb, txt_emb, similarity_score, texts, uids in zip(image_embeddings, text_embeddings, similarity_scores, texts, uids):
                embeddings_list.append({"image_embedding": img_emb.numpy(), 
                                   "text_embedding": txt_emb.numpy(), 
                                   "similarity_score": similarity_score.numpy(),
                                   "text": texts, 
                                   "uid": uids})
        i+=1
        if i>3:
            return embeddings_list

def fix_embedding(embedding):
    dict_embed = {}
    for e in embedding:
        for key, value in e.items():
            if key not in dict_embed:
                dict_embed[key] = []
            dict_embed[key].append(value)
    return dict_embed

embedding=get_embeddings_1(dataloader, clip_processor, clip_model)

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


preprocessed
preprocessed
preprocessed


KeyboardInterrupt: 

In [ ]:
file_path = '/data/vision/beery/scratch/neha/task-datacomp/all_datasets/COOS/COOS7_train.hdf5'
data_file = h5py.File(file_path, 'r')
# print(type(data_file['data'][:]))
total_samples = len(data_file['data'])
min_val = np.max(data_file['data'][:])
print(min_val)

In [ ]:
dataset=COOSDataset('train')
img, text, label, uid = dataset[89]
plt.imshow(img.permute(1,2,0))

In [ ]:
#### embed=np.load('/data/vision/beery/scratch/neha/task-datacomp/all_datasets/embeddings/COOS_train_normalemebed.npy',allow_pickle=True)

###### print(embed[0])

In [ ]:
embedding = {}
for e in embed:
    for key, value in e.items():
        if key not in embedding:
            embedding[key] = []
        embedding[key].append(value)

In [ ]:
len(embedding['image_embedding'])

In [ ]:
embedding.keys()

In [ ]:
centroids=np.load('/data/vision/beery/scratch/neha/task-datacomp/all_datasets/embeddings/centroids.npy')
centroids

In [ ]:
embed = [e["image_embedding"] for e in embed]
embed[0].shape

In [ ]:
np.vstack(embed).shape

In [ ]:
dataset=FMoWDataset('train')
img, text, label, uid = dataset[23]
plt.imshow(img.permute(1,2,0))

In [ ]:
embedding_new=fix_embedding(embedding)

In [ ]:
len(embedding_new['image_embedding'])